# Figure 1: Host DNA percentage

## A: Human host DNA percentage

Importing libraries

In [50]:
import multiprocessing
import pandas as pd
import subprocess
import pysam
import os
from tqdm import tqdm
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re
import dill

### Summary of BAM files generate by [Anonymap](https://github.com/maxibor/anonymap)

|     Lab    |    Contact    | Origin of samples | Number of samples | Reference human genome used | Westernized ?|
|:----------:|:-------------:|:-----------------:|:-----------------:|:---------------------------:|:-------------|
| Kostic Lab | Marsha Wibowo |         ?         |         49        |            GRCh38           |      yes     |
|  Lewis Lab |  Tanvi Honap  |    Burkina Faso   |         69        |             hg19            |       no     | 

Setting up the list of bam files

In [35]:
human_western_bam_dir = "../../data/lewis_lab"
human_western_bams = [i for i in os.listdir(human_western_bam_dir) if i.endswith(".bam")]
human_non_western_bam_dir = "../../data/kostic_lab" 
human_non_western_bams = [i for i in os.listdir(human_non_western_bam_dir) if i.endswith(".bam")]

#### Handling non westernized bam files 

Defining functions to parse alignment files

In [24]:
def bam_index(bam, bamdir):
    print(bam)
    cmd = f"samtools index {bamdir}/{bam}"
    try:
        print(cmd)
        subprocess.check_output(cmd, shell=True)
    except subprocess.CalledProcessError:
        print(f"Error indexing {bam}")

In [25]:
def perChromosome(chr, bam, min_id, min_len):
    resdic = {}
    mapped = 0
    bamfile = pysam.AlignmentFile(bam, "rb")
    reads = bamfile.fetch(chr, multiple_iterators=True)
    for read in reads:
        mismatch = read.get_tag("NM")
        alnLen = read.query_alignment_length
        readLen = read.query_length
        identity = (alnLen - mismatch) / readLen
        if identity >= min_id and alnLen > min_len :
            mapped +=1
    return(mapped)

def getNumberMappedReadsMultiprocess(bam, processes, min_id, min_len):
    try:
        bamfile = pysam.AlignmentFile(bam, "rb")
    except ValueError as e:
        print(e)
    chrs = bamfile.references

    perChromosomePartial = partial(
        perChromosome, bam=bam, min_id = min_id, min_len = min_len)
    p = multiprocessing.Pool(processes)
    result = p.map(perChromosomePartial, chrs)
    p.close()
    p.join()
    mapped = sum(result)
    total = bamfile.mapped + bamfile.unmapped
    return({bam.split("/")[-1].split(".")[0]:[mapped, total]})

def getTotalReads(bam):
    try:
        bamfile = pysam.AlignmentFile(bam, "rb")
    except ValueError as e:
        print(e)
    total = bamfile.mapped + bamfile.unmapped
    return({bam.split("/")[-1].split(".")[0]:total})
        

In [66]:
index_non_western = partial(bam_index, bamdir = human_non_western_bam_dir)
with multiprocessing.Pool(10) as p:
    result_non_western = p.map(index_non_western, human_non_western_bams)

E5.anonym.bam
E2.anonym.bam
A8.anonym.bam
E12.anonym.bam
C2.anonym.bam
B10.anonym.bam
D1.anonym.bam
A2.anonym.bam
C9.anonym.bam
samtools index ../../data/kostic_lab/E5.anonym.bam
B7.anonym.bam
samtools index ../../data/kostic_lab/A8.anonym.bam
samtools index ../../data/kostic_lab/C2.anonym.bam
samtools index ../../data/kostic_lab/E2.anonym.bam
samtools index ../../data/kostic_lab/B10.anonym.bam
samtools index ../../data/kostic_lab/E12.anonym.bam
samtools index ../../data/kostic_lab/A2.anonym.bam
samtools index ../../data/kostic_lab/C9.anonym.bam
samtools index ../../data/kostic_lab/B7.anonym.bam
samtools index ../../data/kostic_lab/D1.anonym.bam
E1.anonym.bam
samtools index ../../data/kostic_lab/E1.anonym.bam
D7.anonym.bam
samtools index ../../data/kostic_lab/D7.anonym.bam


Process ForkPoolWorker-176:
Process ForkPoolWorker-172:
Process ForkPoolWorker-177:
Process ForkPoolWorker-173:
Process ForkPoolWorker-178:
Process ForkPoolWorker-174:
Process ForkPoolWorker-175:
Process ForkPoolWorker-180:
Process ForkPoolWorker-179:
Process ForkPoolWorker-171:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/projects1/users/borry/15_miniconda3/envs/coproid_article/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/projects1/users/borry/15_miniconda3/envs/coproid_article/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/projects1/users/borry/15_miniconda3/envs/coproid_article/lib/pyth

KeyboardInterrupt: 

#### Handling westernized bam files 

Because these files were anonymized for the mapping position as well, we can not index them and have to parse them manually from sam files

In [36]:
def bam2sam(bamfile, bamdir):
    print(bamfile)
    basename = bamfile.split(".")[0]
    cmd = f"samtools view {bamdir}/{bamfile} > {bamdir}/{basename}.sam"
    print(cmd)
    subprocess.check_output(cmd, shell = True)

In [37]:
western2sam = partial(bam2sam, bamdir = human_western_bam_dir)
with multiprocessing.Pool(10) as p:
    p.map(western2sam, human_western_bams)

bftm270223.anonym.bam
bftm0902ecol40.anonym.bam
bftm2103ext32.anonym.bam
bftm080215.anonym.bam
bftm280135.anonym.bam
bftm2601ext24.anonym.bam
bftm2002ext29.anonym.bam
bftm080321.anonym.bam
bftm190220.anonym.bam
samtools view ../../data/lewis_lab/bftm270223.anonym.bam > ../../data/lewis_lab/bftm270223.sam
samtools view ../../data/lewis_lab/bftm0902ecol40.anonym.bam > ../../data/lewis_lab/bftm0902ecol40.sam
samtools view ../../data/lewis_lab/bftm080215.anonym.bam > ../../data/lewis_lab/bftm080215.sam
bftm030236.anonym.bam
samtools view ../../data/lewis_lab/bftm280135.anonym.bam > ../../data/lewis_lab/bftm280135.sam
samtools view ../../data/lewis_lab/bftm2103ext32.anonym.bam > ../../data/lewis_lab/bftm2103ext32.sam
samtools view ../../data/lewis_lab/bftm2601ext24.anonym.bam > ../../data/lewis_lab/bftm2601ext24.sam
samtools view ../../data/lewis_lab/bftm190220.anonym.bam > ../../data/lewis_lab/bftm190220.sam
samtools view ../../data/lewis_lab/bftm2002ext29.anonym.bam > ../../data/lewis_lab

In [41]:
regex = re.compile("(\d*)M")
def samline(sam, regex=regex, min_id = 0.95, min_len=20):
    cnt = 0
    all_cnt = 0
    print(sam)
    with open(sam, 'r') as f:
        for l in f:
            lsplit = l.split()
            cigar = lsplit[5]
            all_cnt +=1
            if len(cigar) > 1:
                nm = int(lsplit[16].split(':')[-1])
                al_len = sum([int(i) for i in re.findall(regex, cigar)])
                read_len = len(lsplit[9])
                ident = (al_len - nm)/read_len
                if ident >= min_id and al_len>min_len:
                    cnt+=1
            else:
                continue
    return({sam:[cnt,all_cnt]})

In [47]:
human_western_sams = [i for i in os.listdir(human_western_bam_dir) if i.endswith(".sam")]

In [53]:
with multiprocessing.Pool(5) as p:
    res = p.map(samline, [f"{human_western_bam_dir}/{s}" for s in human_western_sams])

../../data/lewis_lab/bftm030236.sam
../../data/lewis_lab/bftm070332.sam
../../data/lewis_lab/bftm140127.sam
../../data/lewis_lab/bftm0401ecol39.sam
../../data/lewis_lab/bftm1003ext22.sam
../../data/lewis_lab/bftm2401ext31.sam
../../data/lewis_lab/bftm1502ecol15.sam
../../data/lewis_lab/bftm230440.sam
../../data/lewis_lab/bftm190220.sam
../../data/lewis_lab/bftm140428.sam
../../data/lewis_lab/bftm130122.sam
../../data/lewis_lab/bftm2301ext33.sam
../../data/lewis_lab/bftm040212.sam
../../data/lewis_lab/bftm280135.sam
../../data/lewis_lab/bftm2103ext32.sam
../../data/lewis_lab/bftm180134.sam
../../data/lewis_lab/bftm100113.sam
../../data/lewis_lab/bftm140341.sam
../../data/lewis_lab/bftm220210.sam
../../data/lewis_lab/bftm0202ext20.sam
../../data/lewis_lab/bftm30016.sam
../../data/lewis_lab/bftm270223.sam
../../data/lewis_lab/bftm1104ecol11.sam
../../data/lewis_lab/bftm2601ext24.sam
../../data/lewis_lab/bftm110237.sam
../../data/lewis_lab/bftm0601ext18.sam
../../data/lewis_lab/bftm0803ext

In [54]:
result = {}
for d in res:
    result.update(d)

In [65]:
l
pd.DataFrame(result).T.index.str.split("/lewis_lab/", expand=True).to_frame().iloc[:,1]

../../data  bftm030236.sam            bftm030236.sam
            bftm1502ecol15.sam    bftm1502ecol15.sam
            bftm130122.sam            bftm130122.sam
            bftm100113.sam            bftm100113.sam
            bftm0401ecol39.sam    bftm0401ecol39.sam
            bftm230440.sam            bftm230440.sam
            bftm2301ext33.sam      bftm2301ext33.sam
            bftm280135.sam            bftm280135.sam
            bftm070332.sam            bftm070332.sam
            bftm2401ext31.sam      bftm2401ext31.sam
            bftm140428.sam            bftm140428.sam
            bftm2103ext32.sam      bftm2103ext32.sam
            bftm140127.sam            bftm140127.sam
            bftm040212.sam            bftm040212.sam
            bftm220210.sam            bftm220210.sam
            bftm110237.sam            bftm110237.sam
            bftm1003ext22.sam      bftm1003ext22.sam
            bftm190220.sam            bftm190220.sam
            bftm180134.sam            bftm1801